In [1]:
import numpy as np
import matplotlib.pylab as plt

In [2]:
from raytracing import rectangle_intersection

In [3]:
def plot_rectangle(uv, lost, rectangle_size, title='rectangle'):
    plt.plot(uv[~lost, 0], uv[~lost, 1], ',b') # inside
    plt.plot(uv[lost, 0], uv[lost, 1], ',r') # lost
    width, height = rectangle_size
    plt.plot([-width/2, width/2, width/2, -width/2, -width/2],
             [-height/2, -height/2, height/2, height/2, -height/2], '-',
             color='black', linewidth=1);
    #plt.axis('equal');
    plt.xlabel('u (mm)'); plt.ylabel('v (mm)')
    plt.title(title);

In [4]:
# test 1 - zero offset
N = 10000
u = np.zeros((N, 3)) - np.array((1, 0, 0))
A = (np.random.rand(N, 3)-.5) + np.array((10, 0, 0))

angles = (np.pi/2, 0, np.pi/4)
offset = (0, 0, 0)
width, height = 1, 1

lost, in_plane_uv, B = rectangle_intersection(A, u, angles, offset, width, height)

plot_rectangle(in_plane_uv, lost, (width, height), title='sample')

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Use of unsupported NumPy function 'numpy.matmul' or unsupported use of the function.

File "raytracing.py", line 38:
def change_base(A, u, angles, offset):
    <source elided>

    A_prime = np.matmul(A, R.T) - offset
    ^

[1] During: typing of get attribute at /home/etiennenavarro/notebooks/diff_X/ray_tracing/raytracing.py (38)

File "raytracing.py", line 38:
def change_base(A, u, angles, offset):
    <source elided>

    A_prime = np.matmul(A, R.T) - offset
    ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba

In [15]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [16]:
%lprun -f rectangle_intersection rectangle_intersection(A, u, angles, offset, width, height)

Timer unit: 1e-06 s

Total time: 0.00417 s
File: /home/etiennenavarro/notebooks/diff_X/ray_tracing/raytracing.py
Function: rectangle_intersection at line 43

Line #      Hits         Time  Per Hit   % Time  Line Contents
    43                                           def rectangle_intersection(A, u, angles, offset, width, height):
    44                                               '''Projection of the ray (A, u) on the plane xy
    45                                                   of the rotated (angles) and translated (offset) base
    46                                               '''
    47         1       1886.0   1886.0     45.2      A_prime, u_prime = change_base(A, u, angles, offset)
    48                                           
    49         1         22.0     22.0      0.5      time_to_plane = -np.divide(A_prime[:, 2], u_prime[:, 2],
    50         1        245.0    245.0      5.9                                 where=u_prime[:, 2] < 0)
    51                    

## Numba

In [5]:
import numpy as np
from numba import jit

'''second version
    use base change
    
'''
@jit(nopython=True)
def rotation_matrix(omega, psi, phi):
    '''Rotation matrix defined with Euler's angles
        i.e. Body Rotation (intrisic) with axis order 'yxz'
        
        angles in radian
    '''
    s_omega, c_omega = np.sin(omega), np.cos(omega)
    s_psi, c_psi = np.sin(psi), np.cos(psi)
    s_phi, c_phi = np.sin(phi), np.cos(phi)

    R = np.array([[ s_omega*s_phi*s_psi + c_omega*c_phi,
                    s_phi*c_psi,
                   -s_omega*c_phi + s_phi*s_psi*c_omega],
                  [ s_omega*s_psi*c_phi - s_phi*c_omega,
                    c_phi*c_psi,
                    s_omega*s_phi + s_psi*c_omega*c_phi],
                  [ s_omega*c_psi,
                   -s_psi,
                    c_omega*c_psi]])
    
    return R

@jit(nopython=True)
def change_base(A, u, angles, offset):
    omega, psi, phi = angles
    offset = np.asarray(offset)

    R = rotation_matrix(omega, psi, phi)

    A_prime = np.matmul(A, R.T) - offset
    u_prime = np.matmul(u, R.T)
    
    return A_prime, u_prime

In [6]:
N = 5000
u = np.zeros((N, 3)) - np.array((1, 0, 0))
A = (np.random.rand(N, 3)-.5) + np.array((10, 0, 0))

angles = (np.pi/4, 0, 0)
offset = (-.3, .5, -1)

In [7]:
%%timeit
Ap, up = change_base(A, u, angles, offset)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Use of unsupported NumPy function 'numpy.matmul' or unsupported use of the function.

File "<ipython-input-5-fe63f10797ca>", line 38:
def change_base(A, u, angles, offset):
    <source elided>

    A_prime = np.matmul(A, R.T) - offset
    ^

[1] During: typing of get attribute at <ipython-input-5-fe63f10797ca> (38)

File "<ipython-input-5-fe63f10797ca>", line 38:
def change_base(A, u, angles, offset):
    <source elided>

    A_prime = np.matmul(A, R.T) - offset
    ^

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numb